<a href="https://colab.research.google.com/github/juliaalmeida13/PPD_Exercises/blob/main/Exercicios_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de Pi por integral utilizando MPI


In [ ]:
%%writefile pi_seq.c

/*
    This program calculates pi.
    In order to do that, we compute the integral of:
    4 / (1 + x * x)
    from 0 to 1.
    The value of this integral is pi.
    Based on Tim Mattson's (11/99) sequential implementation.
*/

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <sys/time.h>

int main(int argc, char *argv[]) {

    if(argc != 2){
        printf("Usage: ./pi_seq N\n");
        printf("N: Number of steps\n");
        exit(-1);
    }

    // variables to measure execution time
    struct timeval time_start;
    struct timeval time_end;

    // get the start time
    gettimeofday(&time_start, NULL);

    int num_steps = atoi(argv[1]);
    double x, sum = 0.0;
    double step = 1.0 / (double) num_steps;

    for(int i = 1; i <= num_steps; i++){
        x = (i - 0.5) * step;
        sum += 4.0 / (1.0 + x * x);
    }

    double pi = step * sum;

    // get the end time
    gettimeofday(&time_end, NULL);

    double exec_time = (double) (time_end.tv_sec - time_start.tv_sec) +
                       (double) (time_end.tv_usec - time_start.tv_usec) / 1000000.0;

    printf("pi with %d steps is %.9lf in %lf seconds\n", num_steps, pi, exec_time);

    return 0;
}

Overwriting pi_seq.c


In [ ]:
# !if [ ! -e mult ]; then mpicc -Wall mult.c -o mult ; fi
! gcc -Wall pi_seq.c -o pi_seq
# Execução com medição de tempo pelo comando time. Observe o tempo total decorrido (real)
! ./pi_seq 1000000000

pi with 1000000000 steps is 3.141592654 in 3.583131 seconds


In [ ]:
%%writefile pi_integral_mpi.c

#include <math.h>
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <unistd.h>
#include <time.h>
#include "mpi.h"

int main(int argc, char *argv[])
{

	int numtasks, rank;
	int result;
	int t;



	// Todos os processos iniciam a biblioteca e determinam seus ranks na aplicação

	result = MPI_Init(&argc,&argv);
	if (result != MPI_SUCCESS) {
		fprintf(stderr,"Erro iniciando MPI: %d\n",result);
		MPI_Abort(MPI_COMM_WORLD, result);
	}
	MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
	MPI_Comm_rank(MPI_COMM_WORLD, &rank);


	if(rank==0) {

	   if(argc != 2){
        printf("Usage: ./pi_seq N\n");
        printf("N: Number of steps\n");
        exit(-1);
     }
    int num_steps = atoi(argv[1]);
    printf ("Vai calcular Pi em %d passos \n",num_steps);


      // Processo mestre deve compor o problema e dividí-lo
			// entre os workers


			// Mestre informa os demais sobre o seu "sub-problema"
//			for (i= ... ; i < ... ; i++)
//				MPI_Send( & buffer , quantidade, MPI_INT, i, 1, MPI_COMM_WORLD);
      // Mestre faz a sua parte do trabalho

      // mestre recolhe os resultaod dos workers

  	for(t=1; t < numtasks; t++)
		;
//	  		MPI_Recv( & variavel, quantidade de dados, MPI_INT, t, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

	} else { // worker

		// Recebem informações sobre a sua parte do problema
//		MPI_Recv (&s_info, sizeof(s_info), MPI_INT, 0, MPI_ANY_TAG, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

		// Worker faz a sua parte do trabalho

		// Worker devole resultado parcial para o mestre
//		MPI_Send( & ...., quantidade , MPI_INT, 0, 1, MPI_COMM_WORLD);
	}

	MPI_Finalize();

	return(0);
}


Writing pi_integral_mpi.c


In [ ]:
# !if [ ! -e mult ]; then mpicc -Wall mult.c -o mult ; fi
! mpicc -Wall pi_integral_mpi.c -o pi_integral_mpi
# Execução com medição de tempo pelo comando time. Observe o tempo total decorrido (real)
! time mpirun --allow-run-as-root -n 4 -host localhost:4 pi_integral_mpi 1000000000

Vai calcular Pi em 1000000000 passos 

real	0m0.423s
user	0m0.147s
sys	0m0.160s


# Teste de primitivas do MPI

No proximo programa, vamos criar um mestre e 4 workers, sendo que o mestre e=deve enviar um número inteiro para cada worker. Os workers devem exibir o que receberam.

In [ ]:
%%writefile teste_primitivas.c

#include <math.h>
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <unistd.h>
#include <time.h>
#include "mpi.h"

int main(int argc, char *argv[])
{

	int numtasks, rank, result, t, buffer, i;

	// Todos os processos iniciam a biblioteca e determinam seus ranks na aplicação

	result = MPI_Init(&argc,&argv);
	if (result != MPI_SUCCESS) {
		fprintf(stderr,"Erro iniciando MPI: %d\n",result);
		MPI_Abort(MPI_COMM_WORLD, result);
	}
	MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
	MPI_Comm_rank(MPI_COMM_WORLD, &rank);


	if(rank==0) {

    printf ("Vai testar o envio de dados para os workers  \n");

		// Mestre envia dados para os workers
  	for (i= 1; i < numtasks; i++) {
      buffer = i * 10;
      MPI_Send( & buffer , 1, MPI_INT, i, 1, MPI_COMM_WORLD);

    }
      // mestre recolhe os resultaod dos workers

  	for(t=1; t < numtasks; t++) {
	  		MPI_Recv( & buffer, 1, MPI_INT, t, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf ("O dado %d foi recebido do worker % d \n",buffer,t);
    }
	}

	else {
		// worker

		// Recebem informações sobre a sua parte do problema
		MPI_Recv (&buffer, 1, MPI_INT, 0, MPI_ANY_TAG, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    printf("Processo %d recebeu %d \n",rank,buffer);
    buffer = buffer * 2;
		MPI_Send( &buffer, 1, MPI_INT, 0, 1, MPI_COMM_WORLD);
	}

	MPI_Finalize();

	return(0);
}


Overwriting teste_primitivas.c


In [ ]:
# !if [ ! -e mult ]; then mpicc -Wall mult.c -o mult ; fi
! mpicc -Wall teste_primitivas.c -o teste_primitivas
# Execução com medição de tempo pelo comando time. Observe o tempo total decorrido (real)
! time mpirun --allow-run-as-root -n 5 -host localhost:5 teste_primitivas

Usage: ./pi_seq N
N: Number of steps
--------------------------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code. Per user-direction, the job has been aborted.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
mpirun detected that one or more processes exited with non-zero status, thus causing
the job to be terminated. The first process to do so was:

  Process name: [[1430,1],0]
  Exit code:    255
--------------------------------------------------------------------------

real	0m1.835s
user	0m0.060s
sys	0m0.096s
